In [ ]:
import cv2
import argparse
import imutils
import time
import scipy.io
# from sklearn.svm import SVC
import skvideo.io
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
from scipy.ndimage import morphology as mp
from skimage.feature import canny
from thundersvm import SVC
from random import sample
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import image
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import mahotas
from imutils.video import VideoStream
from imutils.video import FPS
import os
import pickle
import copy

In [ ]:
img = cv2.imread('input/frame.png');
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img=img[:,:,0]
sil = cv2.imread('input/gt.png');
# sil = cv2.cvtColor(sil, cv2.COLOR_BGR2HSV)
sil=sil[:,:,0]
# [m,n] = img.shape
# [m1,n1] = sil.shape
# a=0
# b=0
# if(m1<m):
#     a = m 
plt.imshow(img),plt.show();
plt.imshow(sil*255),plt.show();
# cv2.imshow("Img",img);
# cv2.imshow("Sil",sil);
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# im = img[0:8,0:8]
patches=[]
GT=[]
patches = image.extract_patches_2d(img, (8, 8))
GT = image.extract_patches_2d(sil, (8, 8))
X=[]
Y=[]
i=0;
psize=8
print("Total Patches:"+str(len(patches)));
while i < len(patches):
#     print("Patch:"+str(i));
#     f1 = fd_histogram(patches[i])
#     f2 = fd_haralick(patches[i])
#     f3 = fd_hu_moments(patches[i])
#     a=f1.tolist()
#     b=np.zeros(len(a))
#     for ind in range(len(a)):
#         b[ind] = a[ind][0]
#     # global_feature = [f1,f2,f3]
#     if max(b)!=0:
#         b = (b - min(b))/max(b)
#     if max(f2)!=0:
#         f2 = (f2 - min(f2))/max(f2)
#     if max(f3)!=0:
#         f3 = (f3 - min(f3))/max(f3)
#     global_feature = np.hstack([b,f2,f3])
#     X.append(global_feature)
    

# for i in range(len(patches)):
    a = patches[i].flatten()
    if(np.count_nonzero(GT[i])>=(psize*psize/2)):
        Y.append(1)
    else:
        Y.append(0)

    X.append(a)
    i=i+1


In [ ]:
svc = SVC(C=10,kernel='rbf',gamma=1/(X[0].shape[0]*np.array(X).var()))
svc.fit(X,Y)

In [ ]:
def trainVideo2(file,boxes,label,filename):
    vs = cv2.VideoCapture(file)
    
    # loop over frames from the video stream
    fno = 1
#     frame_array = []
    disp_flag=False
    capture=True
    next_frame=[]
    corners=[]
    img=[]
    storepath = 'output/'+str(label)+"/"
    nos = 100
    if not os.path.exists(storepath):
        os.mkdir(storepath)
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    fname = filename.strip('.mp4')
#     out = cv2.VideoWriter(storepath+fname+'.avi',fourcc, 10.0, (400,500))
    writer = skvideo.io.FFmpegWriter(storepath+fname+"_"+str(nos)+".mp4")
#     out = cv2.VideoWriter(storepath+filename,0x00000021, 10, (400,500))
    
    def mouse_click(event,x,y,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            corners.append([x,y])
            for i in corners:
                [x,y] = i
                cv2.circle(cropped_im,(x,y),3,255,-1)

#     cv2.namedWindow('Frame')
#     cv2.setMouseCallback("Frame", mouse_click)
    cur_no=0
    count = 1
    while True:
        # grab the current frame, then handle if we are using a
        # VideoStream or VideoCapture object
        frame1 = vs.read()
        frame = frame1[1] # if args.get("video", False) else frame1
        # frame = fgbg.apply(frame)

        # check to see if we have reached the end of the stream
        if frame is None:
            break

        # resize the frame (so we can process it faster) and grab the
        # frame dimensions
        frame = imutils.resize(frame, width=500)
        (H, W) = frame.shape[:2]

        # check to see if we are currently tracking an object
        if fno in boxes:
            (x1,y1,x2,y2) = boxes[fno]

            if x2<x1:
                temp=x2
                x2=x1
                x1=temp
            if y2<y1:
                temp=y2
                y2=y1
                y1=temp

            if x2>W:
                x2=W
            if y2>H:
                y2=H
            if x1<0:
                x1=0
            if y1<0:
                y1=0
            cropped_im = frame[y1:y2,x1:x2]
                
            # show the output frame
            key = cv2.waitKey(1) & 0xFF
            
#         # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
            if key == ord('p'):
                img = copy.copy(cropped_im)
                while True:

                    key2 = cv2.waitKey(1) or 0xff
                    cv2.imshow('Frame',cropped_im)

                    if key2 == ord('p'):
                        disp_flag=True
                        break
            if disp_flag:

                mask = np.zeros((img.shape[0], img.shape[1]))
                cv2.fillConvexPoly(mask, np.array(corners), 1)
                cv2.imwrite('frame.png',img)
                cv2.imwrite('gt.png',mask)
#                 cv2.imshow("Mask:", mask)
                mask = mask.astype(np.bool)

                silhouette = np.zeros_like(img)
                silhouette[mask] = img[mask]
            
                 # delete zero columns
#                 silhouette= np.delete(silhouette,np.where(~silhouette.any(axis=0))[0], axis=1)
#                 # delete zero rows
#                 silhouette= np.delete(silhouette,np.where(~silhouette.any(axis=1))[0], axis=0)

                cv2.imwrite('extracted_silhouette.png',silhouette)
                disp_flag=False
                capture = True
#             [m,n,p] = cropped_im.shape
#             diff = 400 - m
#             top = int(diff/2)
#             bot = diff -top
#             diff = 500 - n
#             left = int(diff/2)
#             right = diff -left
#             constant= cv2.copyMakeBorder(cropped_im,top,bot,left,right,cv2.BORDER_CONSTANT,value=[0,0,0])
#             cv2.imshow("Frame", constant)
#             out.write(constant)
#             frame_array.append(constant)
            print("Frame:"+str(fno));
#             plt.imshow(cropped_im),plt.show();
            if(capture):
                next_img = predictFromHSV_Silhouette2(cropped_im)
                [m,n,p] = next_img.shape
                diff = 400 - m
                top = int(diff/2)
                bot = diff -top
                diff = 500 - n
                left = int(diff/2)
                right = diff -left
                padded_img= cv2.copyMakeBorder(next_img,top,bot,left,right,cv2.BORDER_CONSTANT,value=[0,0,0])
                padded_img = cv2.cvtColor(padded_img, cv2.COLOR_BGR2RGB)
                writer.writeFrame(padded_img)
#                 out.write(padded_img)
#                 plt.imshow(padded_img),plt.show();
                cur_no = cur_no+1
                if(cur_no>=nos):
                    count=count+1
                    nos=100*count
#                     capture=False
                    writer.close()
                    writer = skvideo.io.FFmpegWriter(storepath+fname+"_"+str(nos)+".mp4")
#                     out.release()
#                     break
#                 cv2.imshow("Silhouette:"+str(fno), padded_img)
#             plt.clf()

        fno=fno+1

    vs.release()
    writer.close()
#     out.release()
    
    # close all windows
#     cv2.destroyAllWindows()
#     data={'X':X,'Y':Y}
    return frame_array

In [ ]:
def predictFromHSV_Silhouette2(next_img):
    
    i=0;
    TEST1=None
    test_img = cv2.cvtColor(next_img, cv2.COLOR_BGR2HSV)
#     next_img = next_img[:,:,0]
#     psize=16
    psize=8
    [m,n,p] = next_img.shape
    while i<(m-psize):
        j=0;
        while j<(n-psize):
            a = copy.copy(test_img[i:i+psize,j:j+psize,0])
            a = a.flatten()
            if TEST1 is None:
                TEST1 = a
            else:
                TEST1 = np.vstack([TEST1,a])
            j=j+int(psize/2)
        i=i+int(psize/2)

    predictions = svc.predict(TEST1)

    i=0;
    k=0;
    [m,n,p] = next_img.shape
    new_mask = np.zeros([m,n,p])
    count = np.zeros([m,n,p])
    while i<(m-psize):
        j=0;
        while j<(n-psize):
            if k < len(predictions):
                if(predictions[k]==1):
                    new_mask[i:i+psize,j:j+psize] = new_mask[i:i+psize,j:j+psize] + 1
                    count[i:i+psize,j:j+psize] = count[i:i+psize,j:j+psize] + 1
                k = k+1
            j=j+int(psize/2)
        i=i+int(psize/2)

    count [count==0] = 1
    reconstructed1 = []
    reconstructed1 = new_mask/count
    ret,thresh1 = cv2.threshold(reconstructed1,0.5,1,cv2.THRESH_BINARY)
    print("Thresholded Mask:");
    plt.imshow(thresh1),plt.show();

#     mask1 = reconstructed1.astype(np.bool)
    mask1 = thresh1.astype(np.bool)
    silhouette1 = np.zeros_like(next_img)
    silhouette1[mask1] = next_img[mask1]
    print("Thresholded Silhouette:");
#     plt.imshow(silhouette1),plt.show();

    gray_sil = cv2.cvtColor(silhouette1, cv2.COLOR_BGR2GRAY)
    connected_mask = undesired_objects(gray_sil)
    
    stacked_img = np.stack((connected_mask,)*3, axis=-1)
    
#     plt.imshow(stacked_img),plt.show();
    
    mask = stacked_img.astype(np.bool)
    silhouette = np.zeros_like(next_img)
    silhouette[mask] = next_img[mask]
    plt.imshow(silhouette),plt.show();
    
#     edges = canny(connected_mask)
#     fill_holes = mp.binary_fill_holes(edges)

#     stacked_img = np.stack((fill_holes,)*3, axis=-1)
#     stacked_img = stacked_img.astype(float)
#     # type(stacked_img)
# #     print("Filled Mask:");
# #     plt.imshow(stacked_img),plt.show();

#     mask2 = stacked_img.astype(np.bool)
#     silhouette2 = np.zeros_like(next_img)
#     silhouette2[mask2] = next_img[mask2]
# #     print("Filled Silhouette:");
#     plt.imshow(silhouette2),plt.show();
    
    return silhouette

In [ ]:
basePath = "../IndianBirds/"
frame_array=[]
for i in range(22,23):
    boxes = {}
    fullPath = basePath+str(i)+"/"
    allfiles = os.listdir(fullPath)
    f = open(fullPath+"bbox-2.pkl","rb")
    boxes=pickle.load(f)
    f.close()
    j=0;
#     box = boxes[allfiles[0]][221]
    for file in allfiles:
#     for file in ["C3wkLDsVUwM.mp4"]:
        if file.endswith(".mp4"):
#             if(j==1):
            frame_array = trainVideo2(fullPath+file,boxes[file],i,file)
            j=j+1
        if (j==1):
            break